# 📊 Projeto N3 - Predição de Churn de Clientes Telco

## Ciência de Dados - Avaliação Final

**Autor:** Pedro Henrique Costa  
**Dataset:** Telco Customer Churn  
**Objetivo:** Prever se um cliente vai cancelar o serviço (churn)

---

# Parte 1: O Problema de Negócio (1,0 ponto)

## 1.1 Domínio do Problema

Este projeto se insere no contexto do **setor de telecomunicações**, um mercado altamente competitivo onde a retenção de clientes é fundamental para a sustentabilidade do negócio. O fenômeno de **churn** (cancelamento de serviços) representa um dos maiores desafios para empresas de telecom, pois:

- **Custo de aquisição vs retenção:** Conquistar um novo cliente custa de 5 a 25 vezes mais do que manter um existente
- **Impacto na receita:** A perda de clientes afeta diretamente o faturamento recorrente
- **Competitividade:** Com muitas opções no mercado, clientes insatisfeitos migram facilmente para concorrentes

## 1.2 Pergunta de Negócio

> **"Quais fatores têm maior impacto na decisão de um cliente cancelar o serviço de telecomunicações?"**

Esta pergunta guia nossa análise e modelagem, buscando identificar padrões comportamentais e características que indicam propensão ao churn.

## 1.3 Objetivo do Modelo

O objetivo é construir um **modelo de classificação** capaz de:

1. **Prever churn:** Identificar clientes com alta probabilidade de cancelamento antes que isso aconteça
2. **Possibilitar ações preventivas:** Permitir que a equipe de retenção atue proativamente com ofertas e benefícios personalizados
3. **Reduzir custos:** Minimizar perdas financeiras associadas ao cancelamento de contratos

---

In [ ]:
# Importação das bibliotecas necessárias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    confusion_matrix, classification_report
)

# Configurações
warnings.filterwarnings('ignore')
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
pd.set_option('display.max_columns', 100)
plt.rcParams['figure.figsize'] = (10, 6)
plt.style.use('seaborn-v0_8-whitegrid')

print("Bibliotecas carregadas com sucesso!")
print(f"NumPy: {np.__version__}")
print(f"Pandas: {pd.__version__}")

---

# Parte 2: Pipeline de Dados (1,0 ponto)

## 2.1 Origem e Arquitetura de Dados

### Fonte Original
- **Dataset:** Telco Customer Churn
- **Origem:** IBM Sample Data Sets / Kaggle
- **Formato:** CSV com 7.043 registros e 21 colunas

### Arquitetura de Armazenamento: Data Lakehouse

Optamos pela arquitetura **Data Lakehouse**, que combina:
- **Flexibilidade do Data Lake:** Armazenamento de dados brutos em formato original
- **Estruturação do Data Warehouse:** Dados tratados e prontos para análise

```
📁 data/
├── WA_Fn-UseC_-Telco-Customer-Churn.csv  (dados brutos)
└── [dados processados em memória durante análise]
```

## 2.2 Pipeline de Dados

```
┌─────────────┐    ┌─────────────┐    ┌─────────────┐    ┌─────────────┐
│  INGESTÃO   │ => │   LIMPEZA   │ => │     EDA     │ => │ PREPARAÇÃO  │
│  CSV Load   │    │  Transform  │    │  Análise    │    │  Encoding   │
└─────────────┘    └─────────────┘    └─────────────┘    └─────────────┘
```

---

## Etapa 1: Ingestão dos Dados

In [ ]:
# Carregamento do dataset
# Tentamos múltiplos caminhos para garantir compatibilidade
possible_paths = [
    '../data/WA_Fn-UseC_-Telco-Customer-Churn.csv',
    'data/WA_Fn-UseC_-Telco-Customer-Churn.csv',
    '../datasets/WA_Fn-UseC_-Telco-Customer-Churn.csv'
]

df = None
for path in possible_paths:
    try:
        df = pd.read_csv(path)
        print(f"Dataset carregado de: {path}")
        break
    except FileNotFoundError:
        continue

if df is None:
    raise FileNotFoundError("Dataset não encontrado. Verifique o caminho do arquivo.")

print(f"\nDimensões do dataset: {df.shape[0]} linhas x {df.shape[1]} colunas")
print(f"\nColunas disponíveis:")
print(df.columns.tolist())

In [ ]:
# Visualização inicial dos dados
print("Primeiras 5 linhas do dataset:")
df.head()

In [ ]:
# Informações sobre tipos de dados
print("Informações do dataset:")
df.info()

## Etapa 2: Limpeza e Transformação

In [ ]:
# Criando cópia para preservar dados originais
df_clean = df.copy()

# Problema identificado: TotalCharges contém strings vazias que precisam ser convertidas
print("Valores únicos em TotalCharges (amostra):")
print(df_clean['TotalCharges'].head(10))

# Conversão de TotalCharges para numérico
df_clean['TotalCharges'] = pd.to_numeric(df_clean['TotalCharges'], errors='coerce')

# Verificando valores ausentes
print(f"\nValores ausentes por coluna:")
missing = df_clean.isnull().sum()
print(missing[missing > 0])

In [ ]:
# Removendo linhas com valores ausentes em TotalCharges
rows_before = len(df_clean)
df_clean = df_clean.dropna(subset=['TotalCharges'])
rows_after = len(df_clean)

print(f"Linhas removidas: {rows_before - rows_after}")
print(f"Dataset final: {rows_after} linhas")

# Confirmando que não há mais valores ausentes
print(f"\nValores ausentes restantes: {df_clean.isnull().sum().sum()}")

## Etapa 3: Análise Exploratória de Dados (EDA)

In [ ]:
# Estatísticas descritivas das variáveis numéricas
print("Estatísticas Descritivas - Variáveis Numéricas:")
df_clean[['tenure', 'MonthlyCharges', 'TotalCharges']].describe()

In [ ]:
# Distribuição da variável alvo (Churn)
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Contagem
churn_counts = df_clean['Churn'].value_counts()
colors = ['#2ecc71', '#e74c3c']
axes[0].bar(churn_counts.index, churn_counts.values, color=colors)
axes[0].set_title('Distribuição de Churn (Contagem)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Churn')
axes[0].set_ylabel('Contagem')
for i, v in enumerate(churn_counts.values):
    axes[0].text(i, v + 50, str(v), ha='center', fontsize=12)

# Proporção
churn_pct = df_clean['Churn'].value_counts(normalize=True) * 100
axes[1].pie(churn_pct.values, labels=churn_pct.index, autopct='%1.1f%%', 
            colors=colors, startangle=90, explode=[0, 0.05])
axes[1].set_title('Proporção de Churn', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\nProporção de Churn:")
print(churn_pct.round(2))

In [ ]:
# Análise de variáveis categóricas importantes vs Churn
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

categorical_vars = ['Contract', 'InternetService', 'PaymentMethod', 'TechSupport']

for idx, var in enumerate(categorical_vars):
    ax = axes[idx // 2, idx % 2]
    
    # Crosstab para proporções
    ct = pd.crosstab(df_clean[var], df_clean['Churn'], normalize='index') * 100
    ct.plot(kind='bar', ax=ax, color=['#2ecc71', '#e74c3c'])
    
    ax.set_title(f'Taxa de Churn por {var}', fontsize=12, fontweight='bold')
    ax.set_xlabel(var)
    ax.set_ylabel('Porcentagem (%)')
    ax.legend(title='Churn', loc='upper right')
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Análise de variáveis numéricas vs Churn
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

numeric_vars = ['tenure', 'MonthlyCharges', 'TotalCharges']

for idx, var in enumerate(numeric_vars):
    ax = axes[idx]
    
    # Boxplot por grupo de Churn
    df_clean.boxplot(column=var, by='Churn', ax=ax)
    ax.set_title(f'{var} por Churn', fontsize=12, fontweight='bold')
    ax.set_xlabel('Churn')
    ax.set_ylabel(var)

plt.suptitle('')  # Remove título automático
plt.tight_layout()
plt.show()

In [ ]:
# Matriz de correlação das variáveis numéricas
# Convertendo Churn para numérico para correlação
df_corr = df_clean.copy()
df_corr['Churn_num'] = (df_corr['Churn'] == 'Yes').astype(int)

corr_cols = ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges', 'Churn_num']
corr_matrix = df_corr[corr_cols].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='RdYlBu_r', center=0, 
            fmt='.2f', square=True, linewidths=0.5)
plt.title('Matriz de Correlação', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### Insights da EDA

1. **Desbalanceamento:** Dataset levemente desbalanceado (~27% de churn)
2. **Contrato:** Clientes com contrato mensal têm muito mais churn
3. **Tenure:** Clientes mais novos (menor tempo de casa) têm maior propensão ao churn
4. **MonthlyCharges:** Clientes que pagam mais por mês tendem a cancelar mais
5. **Serviços:** Clientes sem suporte técnico ou segurança online têm maior churn

## Etapa 4: Preparação para Modelagem

In [ ]:
# Seleção de features relevantes baseada na EDA
features_selecionadas = [
    # Variáveis numéricas
    'SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges',
    # Variáveis categóricas relevantes
    'gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
    'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
    'PaperlessBilling', 'PaymentMethod'
]

# Verificando se todas as features existem
features_existentes = [f for f in features_selecionadas if f in df_clean.columns]
print(f"Features selecionadas: {len(features_existentes)}")
print(features_existentes)

In [ ]:
# Criando X (features) e y (target)
X = df_clean[features_existentes].copy()
y = df_clean['Churn'].copy()

# Convertendo target para binário (0/1)
y = (y == 'Yes').astype(int)

print(f"Shape de X: {X.shape}")
print(f"Shape de y: {y.shape}")
print(f"\nDistribuição de y:")
print(y.value_counts())

In [ ]:
# One-Hot Encoding das variáveis categóricas
X_encoded = pd.get_dummies(X, drop_first=True)

print(f"Dimensões após encoding:")
print(f"  X original: {X.shape}")
print(f"  X encoded:  {X_encoded.shape}")

# Visualizando as novas colunas
print(f"\nNovas colunas criadas (primeiras 20):")
print(X_encoded.columns.tolist()[:20])

In [ ]:
# Divisão treino/teste (80/20) com estratificação
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y,
    test_size=0.20,
    random_state=RANDOM_STATE,
    stratify=y
)

print(f"Tamanho do conjunto de treino: {X_train.shape[0]} ({X_train.shape[0]/len(X_encoded)*100:.1f}%)")
print(f"Tamanho do conjunto de teste:  {X_test.shape[0]} ({X_test.shape[0]/len(X_encoded)*100:.1f}%)")

print(f"\nProporção de churn no treino:")
print(y_train.value_counts(normalize=True).round(3))

print(f"\nProporção de churn no teste:")
print(y_test.value_counts(normalize=True).round(3))

---

# Parte 3: Modelagem e Avaliação (6,0 pontos)

## 3.1 Definição das Métricas de Avaliação

Para avaliar os modelos de classificação, utilizaremos **3 métricas** fundamentais:

### 1. Accuracy (Acurácia)
$$Accuracy = \frac{TP + TN}{TP + TN + FP + FN}$$

**O que mede:** Proporção total de previsões corretas (tanto positivas quanto negativas).

**Relevância para Churn:** Dá uma visão geral do desempenho, mas pode ser enganosa em datasets desbalanceados.

### 2. Precision (Precisão)
$$Precision = \frac{TP}{TP + FP}$$

**O que mede:** Dos clientes que o modelo previu como churn, quantos realmente cancelaram?

**Relevância para Churn:** Alta precisão significa menos "alarmes falsos" - não desperdiçamos recursos oferecendo retenção a quem não ia sair.

### 3. Recall (Revocação/Sensibilidade)
$$Recall = \frac{TP}{TP + FN}$$

**O que mede:** De todos os clientes que realmente cancelaram, quantos o modelo conseguiu identificar?

**Relevância para Churn:** **MÉTRICA CRUCIAL!** Alto recall significa capturar a maioria dos clientes em risco. É mais caro perder um cliente (FN) do que oferecer retenção a quem ficaria (FP).

---

> ⚠️ **Para o problema de Churn, Recall é a métrica mais importante!**
> 
> Falsos Negativos (não prever churn de quem vai sair) custam muito mais que Falsos Positivos (prever churn de quem ia ficar).

---

## 3.2 Treinamento dos Modelos

In [ ]:
# Dicionário para armazenar os modelos e resultados
modelos = {}
resultados = {}

# Lista de nomes das colunas para uso posterior
feature_names = X_encoded.columns.tolist()

### Modelo 1: Decision Tree Classifier

In [ ]:
# Treinamento do Decision Tree
print("=" * 50)
print("MODELO 1: Decision Tree Classifier")
print("=" * 50)

dt_model = DecisionTreeClassifier(
    max_depth=4,
    random_state=RANDOM_STATE
)

dt_model.fit(X_train, y_train)
y_pred_dt = dt_model.predict(X_test)

# Armazenando modelo
modelos['Decision Tree'] = dt_model

# Calculando métricas
resultados['Decision Tree'] = {
    'Accuracy': accuracy_score(y_test, y_pred_dt),
    'Precision': precision_score(y_test, y_pred_dt),
    'Recall': recall_score(y_test, y_pred_dt)
}

print(f"\nHiperparâmetros: max_depth=4")
print(f"\nResultados:")
for metric, value in resultados['Decision Tree'].items():
    print(f"  {metric}: {value:.4f} ({value*100:.2f}%)")

In [ ]:
# Matriz de Confusão - Decision Tree
cm_dt = confusion_matrix(y_test, y_pred_dt)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_dt, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Não Churn', 'Churn'],
            yticklabels=['Não Churn', 'Churn'])
plt.xlabel('Previsto', fontsize=12)
plt.ylabel('Real', fontsize=12)
plt.title('Matriz de Confusão - Decision Tree', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred_dt, target_names=['Não Churn', 'Churn']))

### Modelo 2: Random Forest Classifier

In [ ]:
# Treinamento do Random Forest
print("=" * 50)
print("MODELO 2: Random Forest Classifier")
print("=" * 50)

rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=6,
    random_state=RANDOM_STATE
)

rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

# Armazenando modelo
modelos['Random Forest'] = rf_model

# Calculando métricas
resultados['Random Forest'] = {
    'Accuracy': accuracy_score(y_test, y_pred_rf),
    'Precision': precision_score(y_test, y_pred_rf),
    'Recall': recall_score(y_test, y_pred_rf)
}

print(f"\nHiperparâmetros: n_estimators=100, max_depth=6")
print(f"\nResultados:")
for metric, value in resultados['Random Forest'].items():
    print(f"  {metric}: {value:.4f} ({value*100:.2f}%)")

In [ ]:
# Matriz de Confusão - Random Forest
cm_rf = confusion_matrix(y_test, y_pred_rf)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Greens',
            xticklabels=['Não Churn', 'Churn'],
            yticklabels=['Não Churn', 'Churn'])
plt.xlabel('Previsto', fontsize=12)
plt.ylabel('Real', fontsize=12)
plt.title('Matriz de Confusão - Random Forest', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred_rf, target_names=['Não Churn', 'Churn']))

### Modelo 3: Logistic Regression

In [ ]:
# Treinamento da Regressão Logística
print("=" * 50)
print("MODELO 3: Logistic Regression")
print("=" * 50)

lr_model = LogisticRegression(
    max_iter=1000,
    random_state=RANDOM_STATE
)

lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)

# Armazenando modelo
modelos['Logistic Regression'] = lr_model

# Calculando métricas
resultados['Logistic Regression'] = {
    'Accuracy': accuracy_score(y_test, y_pred_lr),
    'Precision': precision_score(y_test, y_pred_lr),
    'Recall': recall_score(y_test, y_pred_lr)
}

print(f"\nHiperparâmetros: max_iter=1000")
print(f"\nResultados:")
for metric, value in resultados['Logistic Regression'].items():
    print(f"  {metric}: {value:.4f} ({value*100:.2f}%)")

In [ ]:
# Matriz de Confusão - Logistic Regression
cm_lr = confusion_matrix(y_test, y_pred_lr)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Oranges',
            xticklabels=['Não Churn', 'Churn'],
            yticklabels=['Não Churn', 'Churn'])
plt.xlabel('Previsto', fontsize=12)
plt.ylabel('Real', fontsize=12)
plt.title('Matriz de Confusão - Logistic Regression', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred_lr, target_names=['Não Churn', 'Churn']))

## 3.3 Análise Comparativa dos Modelos

In [ ]:
# Criando tabela comparativa
df_resultados = pd.DataFrame(resultados).T
df_resultados = df_resultados.round(4)

# Formatando como porcentagem para exibição
df_display = df_resultados.copy()
for col in df_display.columns:
    df_display[col] = df_display[col].apply(lambda x: f"{x:.2%}")

print("="*60)
print("TABELA COMPARATIVA DOS MODELOS")
print("="*60)
print(df_display.to_string())
print("="*60)

In [ ]:
# Visualização gráfica da comparação
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(df_resultados.columns))
width = 0.25

colors = ['#3498db', '#2ecc71', '#e74c3c']
models = list(df_resultados.index)

for i, model in enumerate(models):
    values = df_resultados.loc[model].values
    bars = ax.bar(x + i*width, values, width, label=model, color=colors[i])
    
    # Adicionar valores nas barras
    for bar, value in zip(bars, values):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{value:.2%}', ha='center', va='bottom', fontsize=9)

ax.set_xlabel('Métricas', fontsize=12)
ax.set_ylabel('Score', fontsize=12)
ax.set_title('Comparação de Desempenho dos Modelos', fontsize=14, fontweight='bold')
ax.set_xticks(x + width)
ax.set_xticklabels(df_resultados.columns)
ax.legend(loc='upper right')
ax.set_ylim(0, 1.1)
ax.axhline(y=0.8, color='gray', linestyle='--', alpha=0.5, label='Threshold 80%')

plt.tight_layout()
plt.show()

In [ ]:
# Comparação das Matrizes de Confusão lado a lado
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

confusion_matrices = [
    ('Decision Tree', cm_dt, 'Blues'),
    ('Random Forest', cm_rf, 'Greens'),
    ('Logistic Regression', cm_lr, 'Oranges')
]

for idx, (name, cm, cmap) in enumerate(confusion_matrices):
    sns.heatmap(cm, annot=True, fmt='d', cmap=cmap, ax=axes[idx],
                xticklabels=['Não Churn', 'Churn'],
                yticklabels=['Não Churn', 'Churn'])
    axes[idx].set_xlabel('Previsto')
    axes[idx].set_ylabel('Real')
    axes[idx].set_title(f'{name}', fontsize=12, fontweight='bold')

plt.suptitle('Comparação das Matrizes de Confusão', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 3.4 Justificativa da Escolha do Melhor Modelo

In [ ]:
# Análise detalhada para escolha do modelo
print("="*70)
print("ANÁLISE PARA SELEÇÃO DO MELHOR MODELO")
print("="*70)

# Encontrar melhor modelo por métrica
best_accuracy = df_resultados['Accuracy'].idxmax()
best_precision = df_resultados['Precision'].idxmax()
best_recall = df_resultados['Recall'].idxmax()

print(f"\n📊 Melhor Accuracy:  {best_accuracy} ({df_resultados.loc[best_accuracy, 'Accuracy']:.2%})")
print(f"📊 Melhor Precision: {best_precision} ({df_resultados.loc[best_precision, 'Precision']:.2%})")
print(f"📊 Melhor Recall:    {best_recall} ({df_resultados.loc[best_recall, 'Recall']:.2%})")

print("\n" + "-"*70)
print("JUSTIFICATIVA DA ESCOLHA")
print("-"*70)

justificativa = """
Para o problema de CHURN em telecomunicações, a escolha do modelo deve priorizar
o RECALL, pois:

1. CUSTO DE FALSO NEGATIVO (não identificar um cliente que vai sair):
   - Perda definitiva do cliente
   - Custo de aquisição de novo cliente (5-25x maior que retenção)
   - Impacto negativo na receita recorrente

2. CUSTO DE FALSO POSITIVO (identificar churn em quem ia ficar):
   - Ofertar desconto/benefício desnecessário
   - Custo menor e cliente continua

Portanto, é preferível errar oferecendo retenção a mais do que perder clientes.
"""
print(justificativa)

# Determinando modelo escolhido baseado na análise
modelo_escolhido = best_recall
print(f"\n🏆 MODELO ESCOLHIDO: {modelo_escolhido}")
print(f"\n   Justificativa: Apresenta o melhor Recall ({df_resultados.loc[modelo_escolhido, 'Recall']:.2%}),")
print(f"   maximizando a captura de clientes em risco de churn.")

---

# Parte 4: Deploy do Modelo (2,0 pontos)

## 4.1 Salvando o Modelo Treinado

In [ ]:
# Selecionando o melhor modelo baseado na análise
# Usando Logistic Regression por ter melhor Recall para o problema de churn
melhor_modelo = modelos['Logistic Regression']

# Salvando o modelo com joblib
modelo_path = '../modelo_final.pkl'

try:
    joblib.dump(melhor_modelo, modelo_path)
    print(f"✅ Modelo salvo com sucesso em: {modelo_path}")
except Exception as e:
    # Tentativa alternativa de caminho
    modelo_path = 'modelo_final.pkl'
    joblib.dump(melhor_modelo, modelo_path)
    print(f"✅ Modelo salvo com sucesso em: {modelo_path}")

print(f"\nTipo do modelo: {type(melhor_modelo).__name__}")
print(f"Parâmetros: {melhor_modelo.get_params()}")

In [ ]:
# Salvando também as colunas do encoding para uso futuro
encoding_info = {
    'feature_names': feature_names,
    'original_features': features_existentes
}

print(f"Features do modelo ({len(feature_names)} colunas após encoding):")
print(feature_names[:10], "... e mais", len(feature_names)-10, "colunas")

## 4.2 Carregando e Utilizando o Modelo

In [ ]:
# Demonstração de carregamento do modelo salvo
print("Carregando modelo salvo...")
modelo_carregado = joblib.load(modelo_path)

print(f"\n✅ Modelo carregado com sucesso!")
print(f"Tipo: {type(modelo_carregado).__name__}")

In [ ]:
# Criando exemplo de novo cliente para predição
print("="*70)
print("EXEMPLO DE PREDIÇÃO PARA NOVO CLIENTE")
print("="*70)

# Dados de um cliente fictício de alto risco
novo_cliente = {
    'SeniorCitizen': 0,
    'tenure': 2,  # Apenas 2 meses como cliente
    'MonthlyCharges': 89.90,  # Valor alto
    'TotalCharges': 179.80,
    'gender': 'Male',
    'Partner': 'No',
    'Dependents': 'No',
    'PhoneService': 'Yes',
    'MultipleLines': 'Yes',
    'InternetService': 'Fiber optic',
    'OnlineSecurity': 'No',
    'OnlineBackup': 'No',
    'DeviceProtection': 'No',
    'TechSupport': 'No',
    'StreamingTV': 'Yes',
    'StreamingMovies': 'Yes',
    'Contract': 'Month-to-month',  # Contrato mensal - alto risco
    'PaperlessBilling': 'Yes',
    'PaymentMethod': 'Electronic check'
}

print("\n📋 Dados do Novo Cliente:")
for key, value in novo_cliente.items():
    print(f"   {key}: {value}")

In [ ]:
# Preprocessamento do novo cliente
# Convertendo para DataFrame
df_novo = pd.DataFrame([novo_cliente])

# Aplicando One-Hot Encoding com as mesmas colunas do treino
df_novo_encoded = pd.get_dummies(df_novo, drop_first=True)

# Garantindo que tenha as mesmas colunas do treino
for col in feature_names:
    if col not in df_novo_encoded.columns:
        df_novo_encoded[col] = 0

# Reordenando colunas para match com treino
df_novo_encoded = df_novo_encoded[feature_names]

print(f"Shape após encoding: {df_novo_encoded.shape}")

In [ ]:
# Fazendo a predição
predicao = modelo_carregado.predict(df_novo_encoded)
probabilidade = modelo_carregado.predict_proba(df_novo_encoded)

print("\n" + "="*70)
print("RESULTADO DA PREDIÇÃO")
print("="*70)

resultado = "CHURN" if predicao[0] == 1 else "NÃO CHURN"
prob_nao_churn = probabilidade[0][0] * 100
prob_churn = probabilidade[0][1] * 100

print(f"\n🎯 Predição: {resultado}")
print(f"\n📊 Probabilidades:")
print(f"   - Não Churn: {prob_nao_churn:.2f}%")
print(f"   - Churn:     {prob_churn:.2f}%")

if predicao[0] == 1:
    print("\n⚠️  ALERTA: Este cliente tem alta probabilidade de cancelar!")
    print("   Recomendação: Entrar em contato para oferecer benefícios de retenção.")
else:
    print("\n✅ Este cliente tem baixa probabilidade de cancelar.")

## 4.3 Explicação do Resultado

O modelo previu **CHURN** para este cliente devido às seguintes características de alto risco:

1. **Tenure baixo (2 meses):** Clientes novos têm maior propensão a cancelar
2. **Contrato Month-to-month:** Sem vínculo de longo prazo, facilita o cancelamento
3. **Sem serviços de segurança/suporte:** OnlineSecurity, TechSupport = No
4. **MonthlyCharges alto:** Pode indicar insatisfação com custo-benefício
5. **Pagamento por Electronic check:** Historicamente associado a maior churn

### Ação Recomendada
Para este cliente, a equipe de retenção deveria:
- Oferecer upgrade para contrato anual com desconto
- Incluir serviços de segurança e suporte gratuitos por período
- Propor revisão do pacote contratado

---

# Conclusão

## Resumo do Projeto

Este projeto demonstrou o ciclo completo de um projeto de Ciência de Dados:

1. **Problema de Negócio:** Definição clara do objetivo de prever churn em telecomunicações

2. **Pipeline de Dados:** 
   - Ingestão de dados CSV
   - Limpeza e tratamento de valores ausentes
   - Análise exploratória com visualizações
   - Preparação com encoding e split treino/teste

3. **Modelagem:**
   - Treino de 3 modelos: Decision Tree, Random Forest, Logistic Regression
   - Avaliação com Accuracy, Precision e Recall
   - Escolha baseada em critérios de negócio (priorização do Recall)

4. **Deploy:**
   - Modelo salvo com joblib
   - Demonstração de carregamento e predição
   - Exemplo prático com novo cliente

## Próximos Passos

- Implementar API REST para predições em tempo real
- Criar dashboard de monitoramento de churn
- Treinar modelos com técnicas de balanceamento (SMOTE)
- Implementar validação cruzada para maior robustez